こちらベースで試した　https://github.com/davabase/whisper_real_time/tree/master

　-rはrequiments.txtでinstallするときの引数

In [2]:
!pip install -r requirements.txt　

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
  Cloning https://github.com/openai/whisper.git to /private/var/folders/z6/9647yzcd6s52_n2gsnrlv9z00000gn/T/pip-req-build-e_qgjgwu
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /private/var/folders/z6/9647yzcd6s52_n2gsnrlv9z00000gn/T/pip-req-build-e_qgjgwu
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
ネットをオフにしても.py版は動く。。インストール時にモデルもDLしているのか？

In [3]:
#! python3.7

import argparse
import os
import numpy as np
import speech_recognition as sr
import whisper
import torch

from datetime import datetime, timedelta
from queue import Queue
from time import sleep
from sys import platform


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", default="medium", help="Model to use",
                        choices=["tiny", "base", "small", "medium", "large"])
    parser.add_argument("--non_english", action='store_true',
                        help="Don't use the english model.")
    parser.add_argument("--energy_threshold", default=1000,
                        help="Energy level for mic to detect.", type=int)
    parser.add_argument("--record_timeout", default=2,
                        help="How real time the recording is in seconds.", type=float)
    parser.add_argument("--phrase_timeout", default=3,
                        help="How much empty space between recordings before we "
                             "consider it a new line in the transcription.", type=float)
    if 'linux' in platform:
        parser.add_argument("--default_microphone", default='pulse',
                            help="Default microphone name for SpeechRecognition. "
                                 "Run this with 'list' to view available Microphones.", type=str)
    args = parser.parse_args()

    # The last time a recording was retrieved from the queue.
    phrase_time = None
    # Thread safe Queue for passing data from the threaded recording callback.
    data_queue = Queue()
    # We use SpeechRecognizer to record our audio because it has a nice feature where it can detect when speech ends.
    recorder = sr.Recognizer()
    recorder.energy_threshold = args.energy_threshold
    # Definitely do this, dynamic energy compensation lowers the energy threshold dramatically to a point where the SpeechRecognizer never stops recording.
    recorder.dynamic_energy_threshold = False

    # Important for linux users.
    # Prevents permanent application hang and crash by using the wrong Microphone
    if 'linux' in platform:
        mic_name = args.default_microphone
        if not mic_name or mic_name == 'list':
            print("Available microphone devices are: ")
            for index, name in enumerate(sr.Microphone.list_microphone_names()):
                print(f"Microphone with name \"{name}\" found")
            return
        else:
            for index, name in enumerate(sr.Microphone.list_microphone_names()):
                if mic_name in name:
                    source = sr.Microphone(sample_rate=16000, device_index=index)
                    break
    else:
        source = sr.Microphone(sample_rate=16000)

    # Load / Download model
    model = args.model
    if args.model != "large" and not args.non_english:
        model = model + ".en"
    audio_model = whisper.load_model(model)

    record_timeout = args.record_timeout
    phrase_timeout = args.phrase_timeout

    transcription = ['']

    with source:
        recorder.adjust_for_ambient_noise(source)

    def record_callback(_, audio:sr.AudioData) -> None:
        """
        Threaded callback function to receive audio data when recordings finish.
        audio: An AudioData containing the recorded bytes.
        """
        # Grab the raw bytes and push it into the thread safe queue.
        data = audio.get_raw_data()
        data_queue.put(data)

    # Create a background thread that will pass us raw audio bytes.
    # We could do this manually but SpeechRecognizer provides a nice helper.
    recorder.listen_in_background(source, record_callback, phrase_time_limit=record_timeout)

    # Cue the user that we're ready to go.
    print("Model loaded.\n")

    while True:
        try:
            now = datetime.utcnow()
            # Pull raw recorded audio from the queue.
            if not data_queue.empty():
                phrase_complete = False
                # If enough time has passed between recordings, consider the phrase complete.
                # Clear the current working audio buffer to start over with the new data.
                if phrase_time and now - phrase_time > timedelta(seconds=phrase_timeout):
                    phrase_complete = True
                # This is the last time we received new audio data from the queue.
                phrase_time = now
                
                # Combine audio data from queue
                audio_data = b''.join(data_queue.queue)
                data_queue.queue.clear()
                
                # Convert in-ram buffer to something the model can use directly without needing a temp file.
                # Convert data from 16 bit wide integers to floating point with a width of 32 bits.
                # Clamp the audio stream frequency to a PCM wavelength compatible default of 32768hz max.
                audio_np = np.frombuffer(audio_data, dtype=np.int16).astype(np.float32) / 32768.0

                # Read the transcription.
                result = audio_model.transcribe(audio_np, fp16=torch.cuda.is_available())
                text = result['text'].strip()

                # If we detected a pause between recordings, add a new item to our transcription.
                # Otherwise edit the existing one.
                if phrase_complete:
                    transcription.append(text)
                else:
                    transcription[-1] = text

                # Clear the console to reprint the updated transcription.
                os.system('cls' if os.name=='nt' else 'clear')
                for line in transcription:
                    print(line)
                # Flush stdout.
                print('', end='', flush=True)
            else:
                # Infinite loops are bad for processors, must sleep.
                sleep(0.25)
        except KeyboardInterrupt:
            break

    print("\n\nTranscription:")
    for line in transcription:
        print(line)


if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] [--model {tiny,base,small,medium,large}]
                             [--non_english]
                             [--energy_threshold ENERGY_THRESHOLD]
                             [--record_timeout RECORD_TIMEOUT]
                             [--phrase_timeout PHRASE_TIMEOUT]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/nori/Library/Jupyter/runtime/kernel-ea600cb3-505b-42c4-93cf-6a42ebc11a28.json


SystemExit: 2

mainを簡単にしてmacで動くようにした


In [ ]:
#! python3.7

import argparse
import os
import numpy as np
import speech_recognition as sr
import whisper
import torch

from datetime import datetime, timedelta
from queue import Queue
from time import sleep
from sys import platform


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", default="base", help="Model to use",
                        choices=["tiny", "base", "small", "medium", "large"])
    parser.add_argument("--non_english", action='store_true',
                        help="Don't use the english model.")
    parser.add_argument("--energy_threshold", default=1000,
                        help="Energy level for mic to detect.", type=int)
    parser.add_argument("--record_timeout", default=0.5, #default 3
                        help="How real time the recording is in seconds.", type=float)
    parser.add_argument("--phrase_timeout", default=0.5, #default 2
                        help="How much empty space between recordings before we "
                             "consider it a new line in the transcription.", type=float)
    if 'linux' in platform:
        parser.add_argument("--default_microphone", default='pulse',
                            help="Default microphone name for SpeechRecognition. "
                                 "Run this with 'list' to view available Microphones.", type=str)
    #args = parser.parse_args()
    args, unknown = parser.parse_known_args()

    
    # The last time a recording was retrieved from the queue.
    phrase_time = None
    # Thread safe Queue for passing data from the threaded recording callback.
    data_queue = Queue()
    # We use SpeechRecognizer to record our audio because it has a nice feature where it can detect when speech ends.
    recorder = sr.Recognizer()
    recorder.energy_threshold = args.energy_threshold
    # Definitely do this, dynamic energy compensation lowers the energy threshold dramatically to a point where the SpeechRecognizer never stops recording.
    recorder.dynamic_energy_threshold = False

    # Important for linux users.
    # Prevents permanent application hang and crash by using the wrong Microphone
    if 'linux' in platform:
        mic_name = args.default_microphone
        if not mic_name or mic_name == 'list':
            print("Available microphone devices are: ")
            for index, name in enumerate(sr.Microphone.list_microphone_names()):
                print(f"Microphone with name \"{name}\" found")
            return
        else:
            for index, name in enumerate(sr.Microphone.list_microphone_names()):
                if mic_name in name:
                    source = sr.Microphone(sample_rate=16000, device_index=index)
                    break
    else:
        source = sr.Microphone(sample_rate=16000)

    # Load / Download model
    model = args.model
    '''
    if args.model != "large" and not args.non_english:
        model = model + ".en"
        '''
    audio_model = whisper.load_model(model)

    record_timeout = args.record_timeout
    phrase_timeout = args.phrase_timeout

    transcription = ['']

    with source:
        recorder.adjust_for_ambient_noise(source)

    def record_callback(_, audio:sr.AudioData) -> None:
        """
        Threaded callback function to receive audio data when recordings finish.
        audio: An AudioData containing the recorded bytes.
        """
        # Grab the raw bytes and push it into the thread safe queue.
        data = audio.get_raw_data()
        data_queue.put(data)

    # Create a background thread that will pass us raw audio bytes.
    # We could do this manually but SpeechRecognizer provides a nice helper.
    recorder.listen_in_background(source, record_callback, phrase_time_limit=record_timeout)

    # Cue the user that we're ready to go.
    print("Model loaded.\n")

    while True:
        try:
            now = datetime.utcnow()
            # Pull raw recorded audio from the queue.
            if not data_queue.empty():
                phrase_complete = False
                # If enough time has passed between recordings, consider the phrase complete.
                # Clear the current working audio buffer to start over with the new data.
                if phrase_time and now - phrase_time > timedelta(seconds=phrase_timeout):
                    phrase_complete = True
                # This is the last time we received new audio data from the queue.
                phrase_time = now
                
                # Combine audio data from queue
                audio_data = b''.join(data_queue.queue)
                data_queue.queue.clear()
                
                # Convert in-ram buffer to something the model can use directly without needing a temp file.
                # Convert data from 16 bit wide integers to floating point with a width of 32 bits.
                # Clamp the audio stream frequency to a PCM wavelength compatible default of 32768hz max.
                audio_np = np.frombuffer(audio_data, dtype=np.int16).astype(np.float32) / 32768.0

                # Read the transcription.
                result = audio_model.transcribe(audio_np, fp16=torch.cuda.is_available())
                text = result['text'].strip()

                # If we detected a pause between recordings, add a new item to our transcription.
                # Otherwise edit the existing one.
                if phrase_complete:
                    transcription.append(text)
                else:
                    transcription[-1] = text

                # Clear the console to reprint the updated transcription.
                os.system('cls' if os.name=='nt' else 'clear')
                for line in transcription:
                    print(line)
                # Flush stdout.
                print('', end='', flush=True)
            else:
                # Infinite loops are bad for processors, must sleep.
                sleep(0.25)
        except KeyboardInterrupt:
            break

    print("\n\nTranscription:")
    for line in transcription:
        print(line)

main()

Model loaded.

But it knew
我要再一個匿人
我要再一個匿人
まにかくロードできてる
我要再一個匿人
まにかくロードできてる
浮かる
我要再一個匿人
まにかくロードできてる
浮かる

我要再一個匿人
まにかくロードできてる
浮かる


我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this
我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this


我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!
我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!

我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everyt

我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.







我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.








我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
Нечки.
我要再一個匿人
まにかくロード

我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
Нечки.
nu waistu 4 6
ラジコで
And arranged considered.


губьет А
어, crew
Oncuy�etu


Thank you.

Probably looks like something to happen, but these are not the doctors but the patients are the ones they're doing and lots of things, yeah they'll be a pain品, but they'll be a pain tools and
ご視聴ありがとうございました
уже здесь
덮어주기가 부症고요 화장실 이�

Şiştük.

A terazakukan livedat z Esperaszty
It does.
我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
Нечки.
nu waistu 4 6
ラジコで
And arranged considered.


губьет А
어, crew
Oncuy�etu


Thank you.

Probably looks like something to happen, but these are not the doctors but the patients are the ones they're doing and lo

我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
Нечки.
nu waistu 4 6
ラジコで
And arranged considered.


губьет А
어, crew
Oncuy�etu


Thank you.

Probably looks like something to happen, but these are not the doctors but the patients are the ones they're doing and lots of things, yeah they'll be a pain品, but they'll be a pain tools and
ご視聴ありがとうございました
уже здесь
덮어주기가 부症고요 화장실 이�

Şiştük.

A terazakukan livedat z Esperaszty
It does.

That's God Chicago-
…
Hehe!
F Sir
there she is
I know






我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
Нечки.
nu waistu 4 6
ラジコで
And arranged considered.


губьет А
어, crew
Oncuy�etu


Thank you.

Probably looks like something to happen, but these are not t

我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
Нечки.
nu waistu 4 6
ラジコで
And arranged considered.


губьет А
어, crew
Oncuy�etu


Thank you.

Probably looks like something to happen, but these are not the doctors but the patients are the ones they're doing and lots of things, yeah they'll be a pain品, but they'll be a pain tools and
ご視聴ありがとうございました
уже здесь
덮어주기가 부症고요 화장실 이�

Şiştük.

A terazakukan livedat z Esperaszty
It does.

That's God Chicago-
…
Hehe!
F Sir
there she is
I know



















我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
Нечки.
nu waistu 4 6
ラジコで
And arranged considered.


губьет А
어, crew
Oncuy�etu


Thank you.

Probably looks like something to happen, but th

我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
Нечки.
nu waistu 4 6
ラジコで
And arranged considered.


губьет А
어, crew
Oncuy�etu


Thank you.

Probably looks like something to happen, but these are not the doctors but the patients are the ones they're doing and lots of things, yeah they'll be a pain品, but they'll be a pain tools and
ご視聴ありがとうございました
уже здесь
덮어주기가 부症고요 화장실 이�

Şiştük.

A terazakukan livedat z Esperaszty
It does.

That's God Chicago-
…
Hehe!
F Sir
there she is
I know
























我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
Нечки.
nu waistu 4 6
ラジコで
And arranged considered.


губьет А
어, crew
Oncuy�etu


Thank you.

Probably looks like something to happen, b

我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
Нечки.
nu waistu 4 6
ラジコで
And arranged considered.


губьет А
어, crew
Oncuy�etu


Thank you.

Probably looks like something to happen, but these are not the doctors but the patients are the ones they're doing and lots of things, yeah they'll be a pain品, but they'll be a pain tools and
ご視聴ありがとうございました
уже здесь
덮어주기가 부症고요 화장실 이�

Şiştük.

A terazakukan livedat z Esperaszty
It does.

That's God Chicago-
…
Hehe!
F Sir
there she is
I know

































我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
Нечки.
nu waistu 4 6
ラジコで
And arranged considered.


губьет А
어, crew
Oncuy�etu


Thank you.

Probably looks like something to 

我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
Нечки.
nu waistu 4 6
ラジコで
And arranged considered.


губьет А
어, crew
Oncuy�etu


Thank you.

Probably looks like something to happen, but these are not the doctors but the patients are the ones they're doing and lots of things, yeah they'll be a pain品, but they'll be a pain tools and
ご視聴ありがとうございました
уже здесь
덮어주기가 부症고요 화장실 이�

Şiştük.

A terazakukan livedat z Esperaszty
It does.

That's God Chicago-
…
Hehe!
F Sir
there she is
I know

































Thank you.





我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
Нечки.
nu waistu 4 6
ラジコで
And arranged considered.


губьет А
어, crew
Oncuy�etu


Thank you.

Probably looks li

我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
Нечки.
nu waistu 4 6
ラジコで
And arranged considered.


губьет А
어, crew
Oncuy�etu


Thank you.

Probably looks like something to happen, but these are not the doctors but the patients are the ones they're doing and lots of things, yeah they'll be a pain品, but they'll be a pain tools and
ご視聴ありがとうございました
уже здесь
덮어주기가 부症고요 화장실 이�

Şiştük.

A terazakukan livedat z Esperaszty
It does.

That's God Chicago-
…
Hehe!
F Sir
there she is
I know

































Thank you.









Bye.



我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
Нечки.
nu waistu 4 6
ラジコで
And arranged considered.


губьет А
어, crew
Oncuy�etu


Thank you.

Proba

我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
Нечки.
nu waistu 4 6
ラジコで
And arranged considered.


губьет А
어, crew
Oncuy�etu


Thank you.

Probably looks like something to happen, but these are not the doctors but the patients are the ones they're doing and lots of things, yeah they'll be a pain品, but they'll be a pain tools and
ご視聴ありがとうございました
уже здесь
덮어주기가 부症고요 화장실 이�

Şiştük.

A terazakukan livedat z Esperaszty
It does.

That's God Chicago-
…
Hehe!
F Sir
there she is
I know

































Thank you.









Bye.






我要再一個匿人
まにかくロードできてる
浮かる


Translation gets turned out torors thanks to Daniel's F提pro
Well, like this

Everything!
¡Mindoli!








Evet şimdi






Thank you.












Thank you.









sadness
Нечки.
nu waistu 4 6
ラジコで
And arranged considered.


губьет А
어, crew
Oncuy�etu


Thank you.

Pr

In [ ]:
上記のmacで動くコードをミニマムにする

In [1]:
#! python3.7

import argparse
import os
import numpy as np
import speech_recognition as sr
import whisper
import torch

from datetime import datetime, timedelta
from queue import Queue
from time import sleep
from sys import platform


def main():

    energy_threshold = 500 # was 1000
    model = "medium" # "tiny", "base", "small", "medium", "large"

    
    # The last time a recording was retrieved from the queue.
    phrase_time = None
    # Thread safe Queue for passing data from the threaded recording callback.
    data_queue = Queue()
    # We use SpeechRecognizer to record our audio because it has a nice feature where it can detect when speech ends.
    recorder = sr.Recognizer()
    recorder.energy_threshold = energy_threshold 
    #recorder.energy_threshold = args.energy_threshold
    # Definitely do this, dynamic energy compensation lowers the energy threshold dramatically to a point where the SpeechRecognizer never stops recording.
    recorder.dynamic_energy_threshold = True

    # Important for linux users.
    # Prevents permanent application hang and crash by using the wrong Microphone

    source = sr.Microphone(sample_rate=16000)

    # Load / Download model
    audio_model = whisper.load_model(model)
    
    record_timeout = 1
    phrase_timeout = 2

    transcription = ['']

    with source:
        recorder.adjust_for_ambient_noise(source)

    def record_callback(_, audio:sr.AudioData) -> None:
        """
        Threaded callback function to receive audio data when recordings finish.
        audio: An AudioData containing the recorded bytes.
        """
        # Grab the raw bytes and push it into the thread safe queue.
        data = audio.get_raw_data()
        data_queue.put(data)

    # Create a background thread that will pass us raw audio bytes.
    # We could do this manually but SpeechRecognizer provides a nice helper.
    recorder.listen_in_background(source, record_callback, phrase_time_limit=record_timeout)

    # Cue the user that we're ready to go.
    print("Model loaded.\n")

    while True:
        try:
            now = datetime.utcnow()
            # Pull raw recorded audio from the queue.
            if not data_queue.empty():
                phrase_complete = False
                # If enough time has passed between recordings, consider the phrase complete.
                # Clear the current working audio buffer to start over with the new data.
                if phrase_time and now - phrase_time > timedelta(seconds=phrase_timeout):
                    phrase_complete = True
                # This is the last time we received new audio data from the queue.
                phrase_time = now
                
                # Combine audio data from queue
                audio_data = b''.join(data_queue.queue)
                data_queue.queue.clear()
                
                # Convert in-ram buffer to something the model can use directly without needing a temp file.
                # Convert data from 16 bit wide integers to floating point with a width of 32 bits.
                # Clamp the audio stream frequency to a PCM wavelength compatible default of 32768hz max.
                audio_np = np.frombuffer(audio_data, dtype=np.int16).astype(np.float32) / 32768.0

                # Read the transcription.
                result = audio_model.transcribe(audio_np, fp16=torch.cuda.is_available(),verbose=False, language="ja")
                text = result['text'].strip()

                # If we detected a pause between recordings, add a new item to our transcription.
                # Otherwise edit the existing one.
                
                if phrase_complete:
                    transcription.append(text)
                else:
                    transcription[-1] = text


                # Clear the console to reprint the updated transcription.
                os.system('cls' if os.name=='nt' else 'clear')
                for line in transcription:
                    print(line)
                # Flush stdout.
                print('', end='', flush=True)
            else:
                # Infinite loops are bad for processors, must sleep.
                sleep(0.5) #was 0.25
        except KeyboardInterrupt:
            break

    print("\n\nTranscription:")
    for line in transcription:
        print(line)

main()

Model loaded.



  0%|                                               | 0/147 [00:05<?, ?frames/s]


100%|█████████████████████████████████████| 358/358 [00:06<00:00, 51.53frames/s]


突破にハモってしまうと思うので、すっくと休んだり。



100%|█████████████████████████████████████| 787/787 [00:08<00:00, 92.38frames/s]

...


突破にハモってしまうと思うので、すっくと休んだり。
やめたりしていいんだと思うようにしています。一つの仕事を続けることだけを真ん中に置くと
生きていることを粗末にしてしまいがちです。僕はそういう経験を何度かしたことがあります。
自分の検査とか、転職だと思ってその仕事についてる人以外が求められているのは
生き続けるだとか、社会にどうにか生き続けることだと思っています。
そのためには、ちゃんと深呼吸はできる。自分だけのいい喫茶店に。
いいレストラン、いいバー、自分だけの絵室だ、創設、ラジオ番組
いつも近くに置いておくことが大切な気がしています
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました!
お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので
んんんんんんんんんんんんんんんん

おやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさい
ご視聴ありがとうございました
ご視聴ありがとうございました!
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました!
とってひとつくらいで滑らかな
お腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてる
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました。
ご視聴ありがとうございました
ご視聴ありがとうございました


100%|█████████████████████████████████████| 659/659 [00:07<00:00, 91.39frames/s]


突破にハモってしまうと思うので、すっくと休んだり。
やめたりしていいんだと思うようにしています。一つの仕事を続けることだけを真ん中に置くと
生きていることを粗末にしてしまいがちです。僕はそういう経験を何度かしたことがあります。
自分の検査とか、転職だと思ってその仕事についてる人以外が求められているのは
生き続けるだとか、社会にどうにか生き続けることだと思っています。
そのためには、ちゃんと深呼吸はできる。自分だけのいい喫茶店に。
いいレストラン、いいバー、自分だけの絵室だ、創設、ラジオ番組
いつも近くに置いておくことが大切な気がしています
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました!
お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので
んんんんんんんんんんんんんんんん

おやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさい
ご視聴ありがとうございました
ご視聴ありがとうございました!
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました!
とってひとつくらいで滑らかな
お腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてる
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました。
ご視聴ありがとうございました
ご視聴ありがとうございました


  0%|                                               | 0/582 [00:03<?, ?frames/s]




Transcription:

突破にハモってしまうと思うので、すっくと休んだり。
やめたりしていいんだと思うようにしています。一つの仕事を続けることだけを真ん中に置くと
生きていることを粗末にしてしまいがちです。僕はそういう経験を何度かしたことがあります。
自分の検査とか、転職だと思ってその仕事についてる人以外が求められているのは
生き続けるだとか、社会にどうにか生き続けることだと思っています。
そのためには、ちゃんと深呼吸はできる。自分だけのいい喫茶店に。
いいレストラン、いいバー、自分だけの絵室だ、創設、ラジオ番組
いつも近くに置いておくことが大切な気がしています
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました!
お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので、お腹が空いたので
んんんんんんんんんんんんんんんん

おやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさいおやすみなさい
ご視聴ありがとうございました
ご視聴ありがとうございました!
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました!
とってひとつくらいで滑らかな
お腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてるお腹が空いてる
ご視聴ありがとうございました
ご視聴ありがとうございました
ご視聴ありがとうございました。
ご視聴ありがとうございま

ご視聴ありがとうございました　をなくす 表示もフレーズ確定のだけを出すようにした。

In [ ]:
#! python3.7

import argparse
import os
import numpy as np
import speech_recognition as sr
import whisper
import torch

from datetime import datetime, timedelta
from queue import Queue
from time import sleep
from sys import platform


def main():

    energy_threshold = 500 # was 1000
    model = "small" # "tiny", "base", "small", "medium", "large"

    
    # The last time a recording was retrieved from the queue.
    phrase_time = None
    # Thread safe Queue for passing data from the threaded recording callback.
    data_queue = Queue()
    # We use SpeechRecognizer to record our audio because it has a nice feature where it can detect when speech ends.
    recorder = sr.Recognizer()
    recorder.energy_threshold = energy_threshold 
    #recorder.energy_threshold = args.energy_threshold
    # Definitely do this, dynamic energy compensation lowers the energy threshold dramatically to a point where the SpeechRecognizer never stops recording.
    recorder.dynamic_energy_threshold = True

    # Important for linux users.
    # Prevents permanent application hang and crash by using the wrong Microphone

    source = sr.Microphone(sample_rate=16000)

    # Load / Download model
    audio_model = whisper.load_model(model)
    
    record_timeout = 0.5
    phrase_timeout = 1

    transcription = ['']

    with source:
        recorder.adjust_for_ambient_noise(source)

    def record_callback(_, audio:sr.AudioData) -> None:
        """
        Threaded callback function to receive audio data when recordings finish.
        audio: An AudioData containing the recorded bytes.
        """
        # Grab the raw bytes and push it into the thread safe queue.
        data = audio.get_raw_data()
        data_queue.put(data)

    # Create a background thread that will pass us raw audio bytes.
    # We could do this manually but SpeechRecognizer provides a nice helper.
    recorder.listen_in_background(source, record_callback, phrase_time_limit=record_timeout)

    # Cue the user that we're ready to go.
    print("Model loaded.\n")

    while True:
        try:
            now = datetime.utcnow()
            # Pull raw recorded audio from the queue.
            if not data_queue.empty():
                phrase_complete = False
                # If enough time has passed between recordings, consider the phrase complete.
                # Clear the current working audio buffer to start over with the new data.
                if phrase_time and now - phrase_time > timedelta(seconds=phrase_timeout):
                    phrase_complete = True
                # This is the last time we received new audio data from the queue.
                phrase_time = now
                
                # Combine audio data from queue
                audio_data = b''.join(data_queue.queue)
                data_queue.queue.clear()
                
                # Convert in-ram buffer to something the model can use directly without needing a temp file.
                # Convert data from 16 bit wide integers to floating point with a width of 32 bits.
                # Clamp the audio stream frequency to a PCM wavelength compatible default of 32768hz max.
                audio_np = np.frombuffer(audio_data, dtype=np.int16).astype(np.float32) / 32768.0

                # Read the transcription.
                result = audio_model.transcribe(audio_np, fp16=torch.cuda.is_available(),verbose=False, language="ja")
                text = result['text'].strip()

                # If we detected a pause between recordings, add a new item to our transcription.
                # Otherwise edit the existing one.
                
                hallucinationTexts = [
                    "ご視聴ありがとうございました",
                    "ご視聴ありがとうございました。",
                    "ご視聴ありがとうございました!",
                    "最後までご視聴ありがとうございました。",
                    "どうも、ご視聴ありがとうございました。",
                    "Thanks for watching!",
                ]
                
                if text in hallucinationTexts:
                    continue
                
                #if phrase_complete:
                print(text)
                '''
                if phrase_complete:
                    transcription.append(text)
                else:
                    transcription[-1] = text
                


                # Clear the console to reprint the updated transcription.
                os.system('cls' if os.name=='nt' else 'clear')
                for line in transcription:
                    print(line)
                # Flush stdout.
                print('', end='', flush=True)
                '''
                
            else:
                # Infinite loops are bad for processors, must sleep.
                sleep(0.25) #was 0.25
        except KeyboardInterrupt:
            break

    print("\n\nTranscription:")
    for line in transcription:
        print(line)

main()

Model loaded.



100%|████████████████████████████████████| 224/224 [00:01<00:00, 120.70frames/s]


もしもしよっきり変えますか


  0%|                                                | 0/51 [00:01<?, ?frames/s]


  0%|                                               | 0/153 [00:01<?, ?frames/s]


...

100%|█████████████████████████████████████| 153/153 [00:01<00:00, 95.19frames/s]


黄色にしてください


100%|█████████████████████████████████████| 153/153 [00:01<00:00, 99.17frames/s]


顔にしてください


  0%|                                               | 0/179 [00:00<?, ?frames/s]